In [10]:
# Required modules
import nltk # python nlp libraries
import pandas as pd  # data analysis and manipulation tool
import re   # for regular expression operation
import string # transform and remove punctuations
import os   # operation system dependent functionality
import csv  # parse csv files

In [11]:
dataframe = pd.read_csv("Coronavirus-tweets.csv", usecols=["Sentiment", "OriginalTweet"]) # get data set
# print (dataframe) # check if retrieved
colsentiment = dataframe.groupby("Sentiment")
print (colsentiment["OriginalTweet"].count())


dfNegative = dataframe.loc[dataframe["Sentiment"] == "Negative"].head(8000)
dfNeutral = dataframe.loc[dataframe["Sentiment"] == "Neutral"].tail(8000)
dfPositive = dataframe.loc[dataframe["Sentiment"] == "Positive"].sample(8000)

df = [ dfNegative, dfNeutral, dfPositive]
dataframe = pd.concat(df)

print (dataframe.count())

# print (dfExtremelyNegative)  # testing purpose only
# print (dfExtremelyPositive.count())  # testing purpose only
# print (dfNegative.count())  # testing purpose only
# print (dfPositive.count())  # testing purpose only
# print (dfNeutral.count())  # testing purpose only

rawData = [list(x) for x in dataframe.values]  # convert to list



Sentiment
Negative    17031
Neutral      8332
Positive    19592
Name: OriginalTweet, dtype: int64
OriginalTweet    24000
Sentiment        24000
dtype: int64


In [12]:
# filter all raw data
def FilterData(tweet):
    # convert all to lowercase
    lowcaseTweet = tweet.lower()
     #  Removing all numbers
    removednumbersTweet = re.sub(r'\d+', '', lowcaseTweet)
    # Remove punctuations
    removepunctuationsTweet = removednumbersTweet.translate(string.maketrans("",""), string.punctuation)
    # Removing white spaces
    removewhitespacesTweet = removepunctuationsTweet.strip()
    # Removing all characters instead a string
    #s = '@#24A-09=wes()&8973o**_##me'  # contains letters 'Awesome' for testing purposes 
   
    # removing all urls
    filteredData = re.sub(r'http\S+', ' ', removewhitespacesTweet)
    filteredData = re.sub(r'[^A-Za-z]', ' ', filteredData)

#     print (filteredData) # testing purposes
    return (filteredData)

     

In [13]:
# normalise filtered data
def tokenizeStr(data_str):
    lst_tokens = []
    tokenizer = nltk.tokenize.TreebankWordTokenizer()
    tokens = tokenizer.tokenize(data_str)
    for each in tokens:
        lst_tokens.append(each)
    return lst_tokens

def RemoveStopWords(lst_data_str):
    lst_stop_word_removed = [i for i in lst_data_str if not i in set_stop_words]
    return lst_stop_word_removed

def StemWords(lst_data_str):
#     stemmer = nltk.stem.PorterStemmer()
#     stemtokens = (stemmer.stem(token) for token in lst_data_str)
#     print (stemtokens)
    lst_stemtokens = []
#     print(lst_data_str)
    lemma = nltk.stem.WordNetLemmatizer()
    
    stemtokens = (lemma.lemmatize(token) for token in lst_data_str)
    
    for each in stemtokens:
        try:
#             print (each)
            lst_stemtokens.append(each.encode('utf-8'))
        except:
            print("exception")
       
    return lst_stemtokens

In [14]:
# Normalise all review data
def TextPreprocess (data_str):
    lst_tokens = tokenizeStr(data_str)
    lst_removed_stpwrd = RemoveStopWords(lst_tokens)
    lst_stem_words = StemWords(lst_removed_stpwrd)
    processedData = " ".join(lst_stem_words)
    return (processedData)

In [15]:
set_stop_words = []
def setStopWords():
    stop_words = nltk.corpus.stopwords.words('english')
    for each in stop_words:
        #     print (each)
        set_stop_words.append(each.encode('utf-8')) # encode because data read u'all', u'why' representing string data

setStopWords()

In [16]:
# Save all data after cleaning and normalising.
processedDataList = []
processedDataList.append(["OriginalTweets", "Sentiment"])
def NormaliseData (tweet):
    tweetStatement = tweet[0]
    sentiment = tweet[1]
#     print (sentiment) # for testing purpose
    normalisedData = FilterData (tweetStatement)
    processedData = TextPreprocess (normalisedData)
    processedDataCategoryList = [processedData, sentiment ]
    processedDataList.append(processedDataCategoryList)

In [17]:
for eachData in rawData:
#     print (eachData)
    NormaliseData (eachData)

In [18]:
# Write review data to csv file
writer = csv.writer(open('processed-data.csv','wb'))
for each in processedDataList:
    writer.writerow(each)